In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt
from tqdm import tqdm

from advshotgun import adversary
from advshotgun.pretrained_models import Basic_CNN
from advshotgun.attacks.white_box import FGSM, randomize_attack
from torchvision import models, datasets, transforms
from torch.utils.data import DataLoader


In [2]:
seed = 2023
torch.manual_seed(seed)
use_cuda = torch.cuda.is_available()
device = torch.device('cuda' if use_cuda else 'cpu')
print(f"device:{device}")

device:cuda


In [3]:
batch_size = 8

mnist_train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
mnist_test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))

num_classes = mnist_train.classes.__len__()
class_logit = torch.eye(num_classes).cuda()

print(f"number of classess:{num_classes}")
print(f"train size:{mnist_train.__len__()}")
print(f"test size:{mnist_test.__len__()}")

train_dataloader = DataLoader(mnist_train, batch_size = batch_size, shuffle=True)
test_dataloader = DataLoader(mnist_test, batch_size = batch_size, shuffle=False)


number of classess:10
train size:60000
test size:10000


In [4]:
model = Basic_CNN.build_Basic_CNN("9920.pth.tar")
model.cuda()
model.eval()

Basic_CNN(
  (conv1_1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv1_2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2_1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2_2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=3136, out_features=200, bias=True)
  (fc2): Linear(in_features=200, out_features=10, bias=True)
)

In [5]:
attack_method = FGSM(eps=0.05)
benchmark_method = randomize_attack(eps = 0.10)

In [8]:
test_dataloader = DataLoader(mnist_test, batch_size = batch_size, shuffle=False)
acc = []
for batch_idx, (x, gt) in tqdm(enumerate(test_dataloader)):
    x = x.cuda()
    gt = gt.cuda()
    pred = model(x)
    pred_logits = pred.argmax(1)
    adv_x = attack_method(x, x, class_logit[gt], model)
    pred = model(adv_x)
    adv_logits = pred.argmax(1)
    acc.append(torch.eq(adv_logits, pred_logits).float().mean())

print(f"accuracy:{torch.stack(acc).mean().item()}")

1250it [00:03, 351.57it/s]

accuracy:0.818399965763092


In [10]:
# test_dataloader = DataLoader(mnist_test, batch_size = batch_size, shuffle=False)
acc = []
for batch_idx, (x, gt) in tqdm(enumerate(test_dataloader)):
    x = x.cuda()
    gt = gt.cuda()
    pred = model(x)
    pred_logits = pred.argmax(1)
    adv_x = benchmark_method(x, x, class_logit[gt], model)
    pred = model(adv_x)
    adv_logits = pred.argmax(1)
    acc.append(torch.eq(adv_logits, pred_logits).float().mean())

print(f"accuracy:{torch.stack(acc).mean().item()}")

1250it [00:01, 625.91it/s]

accuracy:0.9842000007629395
